# Meta Methods

In [36]:
# Import libraries 
import pandas as pd   # Optional: good package for manipulating data 
import sklearn  # Package with learning algorithms implemented

dfTrain = pd.read_csv(r'..\Datasets\training-dataset-reduced.csv')
dfTest = pd.read_csv(r'..\Datasets\testing-dataset-reduced.csv')

#Remove id

dfTrain = dfTrain.iloc[: , 1:]
dfTest = dfTest.iloc[: , 1:]
X_train, y_train = dfTrain.drop('y', axis=1), dfTrain["y"]
X_test, y_test = dfTest.drop('y', axis=1), dfTest["y"]

display(dfTrain.head())
display(dfTest.head())

,age,campaign,cons.conf.idx,cons.price.idx,contact,day_of_week,default_unknown,default_yes,duration,education_basic.6y,...,marital_married,marital_single,marital_unknown,month,nr.employed,pdays,poutcome_nonexistent,poutcome_success,previous,y
0,0.419753,0.036364,0.602510,0.698753,1.0,0.75,0.0,0.0,0.707998,0.0,...,1.0,0.0,0.0,0.222222,0.859735,1.0,1.0,0.0,0.0,0.0
1,0.246914,0.127273,0.338912,0.669135,0.0,0.75,0.0,0.0,0.427908,0.0,...,1.0,0.0,0.0,0.444444,1.000000,1.0,1.0,0.0,0.0,0.0
2,0.185185,0.000000,0.602510,0.698753,1.0,0.25,1.0,0.0,0.667382,1.0,...,1.0,0.0,0.0,0.222222,0.859735,1.0,1.0,0.0,0.0,0.0
3,0.407407,0.000000,0.376569,0.882307,1.0,0.75,0.0,0.0,0.525348,0.0,...,1.0,0.0,0.0,0.333333,1.000000,1.0,1.0,0.0,0.0,0.0
4,0.370370,0.054545,0.418410,0.296960,0.0,0.00,0.0,0.0,0.646660,0.0,...,0.0,1.0,0.0,0.333333,0.425709,1.0,1.0,0.0,0.0,1.0


,age,campaign,cons.conf.idx,cons.price.idx,contact,day_of_week,default_unknown,default_yes,duration,education_basic.6y,...,marital_married,marital_single,marital_unknown,month,nr.employed,pdays,poutcome_nonexistent,poutcome_success,previous,y
0,0.098765,0.072727,0.154812,0.340608,0.0,0.0,0.0,0.0,0.638682,0.0,...,0.0,0.0,0.0,0.111111,0.512287,1.0,1.0,0.0,0.0,1.0
1,0.209877,0.000000,0.192469,0.269680,0.0,1.0,0.0,0.0,0.475605,0.0,...,0.0,1.0,0.0,0.222222,0.512287,1.0,1.0,0.0,0.0,0.0
2,0.518519,0.000000,0.602510,0.698753,1.0,0.0,1.0,0.0,0.378653,0.0,...,1.0,0.0,0.0,0.222222,0.859735,1.0,1.0,0.0,0.0,0.0
3,0.296296,0.018182,0.192469,0.269680,0.0,1.0,1.0,0.0,0.558171,0.0,...,1.0,0.0,0.0,0.222222,0.512287,1.0,1.0,0.0,0.0,0.0
4,0.395062,0.000000,0.602510,0.698753,1.0,1.0,0.0,0.0,0.579618,0.0,...,0.0,0.0,0.0,0.222222,0.859735,1.0,1.0,0.0,0.0,0.0


## Random Forest

Now we are going to execute the random forest algorithm and see the accuracy and f-measure.

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from statsmodels.stats.proportion import proportion_confint
import matplotlib.pyplot as plt

clf = RandomForestClassifier(n_estimators=200)
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)

# Obtain accuracy score of learned classifier on test data
print(clf.score(X_test, y_test))
print(confusion_matrix(y_test, pred))
print()
print("Accuracy:", sklearn.metrics.accuracy_score(y_test, pred))
print()
print(sklearn.metrics.classification_report(y_test, pred))

0.8667866786678667
[[2564  417]
 [  27  325]]

Accuracy: 0.8667866786678667

              precision    recall  f1-score   support

         0.0       0.99      0.86      0.92      2981
         1.0       0.44      0.92      0.59       352

    accuracy                           0.87      3333
   macro avg       0.71      0.89      0.76      3333
weighted avg       0.93      0.87      0.89      3333



Now we are going to try with ExtraTreesClassifier in order to introduce more randomness in the random trees.

In [47]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(n_estimators=200)
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)

# Obtain accuracy score of learned classifier on test data
print(clf.score(X_test, y_test))
print(confusion_matrix(y_test, pred))
print()
print("Accuracy:", sklearn.metrics.accuracy_score(y_test, pred))
print()
print(sklearn.metrics.classification_report(y_test, pred))

0.8583858385838584
[[2558  423]
 [  49  303]]

Accuracy: 0.8583858385838584

              precision    recall  f1-score   support

         0.0       0.98      0.86      0.92      2981
         1.0       0.42      0.86      0.56       352

    accuracy                           0.86      3333
   macro avg       0.70      0.86      0.74      3333
weighted avg       0.92      0.86      0.88      3333



## Boosting

In [48]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)

# Obtain accuracy score of learned classifier on test data
print(clf.score(X_test, y_test))
print(confusion_matrix(y_test, pred))
print()
print("Accuracy:", sklearn.metrics.accuracy_score(y_test, pred))
print()
print(sklearn.metrics.classification_report(y_test, pred))

0.8751875187518752
[[2615  366]
 [  50  302]]

Accuracy: 0.8751875187518752

              precision    recall  f1-score   support

         0.0       0.98      0.88      0.93      2981
         1.0       0.45      0.86      0.59       352

    accuracy                           0.88      3333
   macro avg       0.72      0.87      0.76      3333
weighted avg       0.93      0.88      0.89      3333



Now we are going to try boosting with decision tree classifiers with depth = 5

In [50]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=200)
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)

# Obtain accuracy score of learned classifier on test data
print(clf.score(X_test, y_test))
print(confusion_matrix(y_test, pred))
print()
print("Accuracy:", sklearn.metrics.accuracy_score(y_test, pred))
print()
print(sklearn.metrics.classification_report(y_test, pred))

0.8586858685868587
[[2550  431]
 [  40  312]]

Accuracy: 0.8586858685868587

              precision    recall  f1-score   support

         0.0       0.98      0.86      0.92      2981
         1.0       0.42      0.89      0.57       352

    accuracy                           0.86      3333
   macro avg       0.70      0.87      0.74      3333
weighted avg       0.92      0.86      0.88      3333

